### Spark SQL

Spark SQL is used to access structured data with Spark.

Access http://localhost:4040 whenever you want to view the jobs execution monitor.

Additional packages can be found at https://spark-packages.org/ (we will use one of these packages to connect to MongoDB).

### Spark Session and SQL Context

In [2]:
from pyspark.sql import Row, SparkSession, SQLContext

In [5]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


**Creating the Spark Session (to work with Spark)**

In [6]:
spSession = SparkSession.builder.master('local').appName('appSparkSql').getOrCreate()

In [8]:
type(spSession)

pyspark.sql.session.SparkSession

**Creating the SQL Context (to work with Spark SQL)**

In [7]:
sqlContext = SQLContext(sc)

In [9]:
type(sqlContext)

pyspark.sql.context.SQLContext

**Importing and Manipulating the Dataset**

In [12]:
rddCarsOriginal = sc.textFile('aux/datasets/cars.csv')

In [13]:
rddCarsOriginal.count()

198

In [14]:
rddCarsNoHeader = rddCarsOriginal.filter(lambda line: "FUELTYPE" not in line)

In [15]:
rddCarsNoHeader.count()

197

In [18]:
rddCarsSplit = rddCarsNoHeader.map(lambda line: line.split(','))

In [39]:
rddCarsRows = rddCarsSplit.map(lambda column: Row( \
    manufacturer = column[0], \
    category = column[4], \
    hp = int(column[7]) \
))

In [40]:
rddCarsRows.collect()

[Row(manufacturer='subaru', category='hatchback', hp=69),
 Row(manufacturer='chevrolet', category='hatchback', hp=48),
 Row(manufacturer='mazda', category='hatchback', hp=68),
 Row(manufacturer='toyota', category='hatchback', hp=62),
 Row(manufacturer='mitsubishi', category='hatchback', hp=68),
 Row(manufacturer='honda', category='hatchback', hp=60),
 Row(manufacturer='nissan', category='sedan', hp=69),
 Row(manufacturer='dodge', category='hatchback', hp=68),
 Row(manufacturer='plymouth', category='hatchback', hp=68),
 Row(manufacturer='mazda', category='hatchback', hp=68),
 Row(manufacturer='mitsubishi', category='hatchback', hp=68),
 Row(manufacturer='dodge', category='hatchback', hp=68),
 Row(manufacturer='plymouth', category='hatchback', hp=68),
 Row(manufacturer='chevrolet', category='hatchback', hp=70),
 Row(manufacturer='toyota', category='hatchback', hp=62),
 Row(manufacturer='dodge', category='hatchback', hp=68),
 Row(manufacturer='honda', category='hatchback', hp=58),
 Row(ma

**Creating a DataFrame from the RDD**

In [41]:
dfCars = spSession.createDataFrame(rddCarsRows)

In [42]:
dfCars.show()

+------------+---------+---+
|manufacturer| category| hp|
+------------+---------+---+
|      subaru|hatchback| 69|
|   chevrolet|hatchback| 48|
|       mazda|hatchback| 68|
|      toyota|hatchback| 62|
|  mitsubishi|hatchback| 68|
|       honda|hatchback| 60|
|      nissan|    sedan| 69|
|       dodge|hatchback| 68|
|    plymouth|hatchback| 68|
|       mazda|hatchback| 68|
|  mitsubishi|hatchback| 68|
|       dodge|hatchback| 68|
|    plymouth|hatchback| 68|
|   chevrolet|hatchback| 70|
|      toyota|hatchback| 62|
|       dodge|hatchback| 68|
|       honda|hatchback| 58|
|      toyota|hatchback| 62|
|       honda|hatchback| 76|
|   chevrolet|    sedan| 70|
+------------+---------+---+
only showing top 20 rows



**Performing Queries on the DataFrame**

In [43]:
dfCars.select('*').show()

+------------+---------+---+
|manufacturer| category| hp|
+------------+---------+---+
|      subaru|hatchback| 69|
|   chevrolet|hatchback| 48|
|       mazda|hatchback| 68|
|      toyota|hatchback| 62|
|  mitsubishi|hatchback| 68|
|       honda|hatchback| 60|
|      nissan|    sedan| 69|
|       dodge|hatchback| 68|
|    plymouth|hatchback| 68|
|       mazda|hatchback| 68|
|  mitsubishi|hatchback| 68|
|       dodge|hatchback| 68|
|    plymouth|hatchback| 68|
|   chevrolet|hatchback| 70|
|      toyota|hatchback| 62|
|       dodge|hatchback| 68|
|       honda|hatchback| 58|
|      toyota|hatchback| 62|
|       honda|hatchback| 76|
|   chevrolet|    sedan| 70|
+------------+---------+---+
only showing top 20 rows



In [45]:
dfCars.orderBy('manufacturer').show()

+------------+-----------+---+
|manufacturer|   category| hp|
+------------+-----------+---+
| alfa-romero|  hatchback|154|
| alfa-romero|convertible|111|
| alfa-romero|convertible|111|
|        audi|      sedan|110|
|        audi|      wagon|110|
|        audi|      sedan|140|
|        audi|      sedan|110|
|        audi|      sedan|115|
|        audi|      sedan|102|
|         bmw|      sedan|121|
|         bmw|      sedan|121|
|         bmw|      sedan|182|
|         bmw|      sedan|182|
|         bmw|      sedan|101|
|         bmw|      sedan|182|
|         bmw|      sedan|121|
|         bmw|      sedan|101|
|   chevrolet|  hatchback| 70|
|   chevrolet|      sedan| 70|
|   chevrolet|  hatchback| 48|
+------------+-----------+---+
only showing top 20 rows



**Creating a Temporary Table from the Data Frame, and Performing Queries**

In [47]:
dfCars.createOrReplaceTempView('tp_cars')

In [49]:
spSession.sql("SELECT * FROM tp_cars WHERE manufacturer = 'nissan'").show()

+------------+---------+---+
|manufacturer| category| hp|
+------------+---------+---+
|      nissan|    sedan| 69|
|      nissan|    sedan| 69|
|      nissan|    sedan| 69|
|      nissan|    sedan| 55|
|      nissan|    sedan| 69|
|      nissan|    wagon| 69|
|      nissan|    sedan| 69|
|      nissan|hatchback| 69|
|      nissan|    wagon| 69|
|      nissan|  hardtop| 69|
|      nissan|hatchback| 97|
|      nissan|    sedan| 97|
|      nissan|    sedan|152|
|      nissan|    sedan|152|
|      nissan|    wagon|152|
|      nissan|hatchback|160|
|      nissan|hatchback|160|
|      nissan|hatchback|200|
+------------+---------+---+



In [57]:
spSession.sql("SELECT \
    manufacturer, category, ROUND(AVG(hp), 2) AS hp_avg \
    FROM tp_cars GROUP BY manufacturer, category \
    ORDER BY hp_avg DESC").show()

+-------------+-----------+------+
| manufacturer|   category|hp_avg|
+-------------+-----------+------+
|      porsche|    hardtop| 207.0|
|      porsche|convertible| 207.0|
|       jaguar|      sedan|204.67|
|      mercury|  hatchback| 175.0|
|mercedes-benz|convertible| 155.0|
|  alfa-romero|  hatchback| 154.0|
|mercedes-benz|    hardtop| 153.5|
|mercedes-benz|      sedan|146.25|
|      porsche|  hatchback| 143.0|
|          bmw|      sedan|138.88|
|       nissan|  hatchback| 137.2|
|        volvo|      wagon| 130.0|
|        volvo|      sedan|127.25|
|         saab|      sedan|126.67|
|         saab|  hatchback|126.67|
|mercedes-benz|      wagon| 123.0|
|       toyota|    hardtop| 116.0|
|       toyota|convertible| 116.0|
|         audi|      sedan| 115.4|
|  alfa-romero|convertible| 111.0|
+-------------+-----------+------+
only showing top 20 rows

